# 情感分类
情感分类任务是需要大模型根据描述的句子给出对应的情感标签，这种指向性很强的目标，就需要进一步对提示词进行约束，在提示词中不仅要包含输入的句子，也要包含标签的种类，否则大模型是不知道具体如何分类的。而在进行大量的实验后，我们发现如果要输出仅输出标签，更加准确，那么可能需要举例表述才能实现。

> 经过实验，`Qwen`模型比较适配中文任务，能很好的理解中文问题，如果是英文，可能会因为大小写出现问题

In [1]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_instruct = '/home/lixinyu/weights/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer_instruct = AutoTokenizer.from_pretrained(model_name_or_path_instruct)
model_instruct = AutoModelForCausalLM.from_pretrained(model_name_or_path_instruct,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.18it/s]


In [2]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

我们先看下英文情感分类

In [3]:
prompt_1="""
Classify the text into neutral, negative or positive. 
Text: I think the food was okay. 
Sentiment:
"""

In [4]:
print(inference(model_instruct,tokenizer_instruct,prompt_1))

{'response': 'Sentiment: Neutral\n\nThe text "I think the food was okay" expresses a mild approval but does not indicate strong feelings either way. Therefore, it is classified as neutral sentiment.', 'generated_tokens_count': 38}


你希望回答的是neutral这样没有其他任何补充的内容，同时你希望它返回的是neutral而不是Neutral，很明显，现在模型的回答一个都不符合。那么如何实现呢？有几种不同的方法可以做到这一点。在这里，你关心的是具体性，所以你给提示词提供的信息越多，结果就越好。你可以尝试提供示例来指定正确的行为。并且对于回答的约束我们需要跟进一步，让我们再试一次.

In [5]:
prompt_2="""
Classify the text into neutral, negative or positive. Only answer the emotional tags.
Text: I think the vacation is okay.
Sentiment: neutral 
Text: I think the food was okay. 
Sentiment:
"""

In [6]:
print(inference(model_instruct,tokenizer_instruct,prompt_2))

{'response': 'neutral', 'generated_tokens_count': 2}


这次模型返回了neutral，这是你所寻找的具体标签。看起来，提示中提供的示例帮助模型在其输出中保持了具体性，并且没有多余的分析内容了，因为你强迫模型必须只能回答标签。
不过上述例子展示了在英文当中的错误，但事实上，在中文方面，Qwen模型要做的更好一点：

In [7]:
prompt_3="""
将文本分为中性、否定或肯定。只回答情感标签。
文本：我觉得这个假期还可以。
标签：中性
文本：这部电影一般
标签：
"""

In [8]:
print(inference(model_instruct,tokenizer_instruct,prompt_3))

{'response': '中性', 'generated_tokens_count': 3}


这或许是因为英文的大小写并不影响含义，而中文就没有这种顾虑，总之，对于Qwen模型，中文的指令遵从效果要好于英文。